In [1]:
import pyspark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
4,application_1571837672077_0005,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [2]:
input_bucket = 's3://nyctaxi5/'
input_path = 'train.csv'
test_path = 'test.csv'
trainingData = spark.read.format("csv").option("header", "true").load(input_bucket+input_path)
#testData = spark.read.format("csv").option("header", "true").load(input_bucket+test_path)
trainingData.show()
#

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

+---------+---------+-------------------+-------------------+---------------+-------------------+------------------+-------------------+------------------+------------------+-------------+
|       id|vendor_id|    pickup_datetime|   dropoff_datetime|passenger_count|   pickup_longitude|   pickup_latitude|  dropoff_longitude|  dropoff_latitude|store_and_fwd_flag|trip_duration|
+---------+---------+-------------------+-------------------+---------------+-------------------+------------------+-------------------+------------------+------------------+-------------+
|id2875421|        2|2016-03-14 17:24:55|2016-03-14 17:32:30|              1|-73.982154846191406|40.767936706542969|-73.964630126953125|40.765602111816406|                 N|          455|
|id2377394|        1|2016-06-12 00:43:35|2016-06-12 00:54:38|              1|-73.980415344238281|40.738563537597656|-73.999481201171875|40.731151580810547|                 N|          663|
|id3858529|        2|2016-01-19 11:35:24|2016-01-19 12:

In [3]:

trainingData = trainingData.drop("pickup_latitude","dropoff_latitude","pickup_datetime","dropoff_datetime","store_and_fwd_flag") #since latitude does not contribute to the duration like longitude

#testData = testData.drop("pickup_latitude","dropoff_latitude","pickup_datetime","dropoff_datetime","store_and_fwd_flag","trip_duration") #since latitude does not contribute to the duration like longitude

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [4]:
print(trainingData.columns)
#print(testData.columns)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

['id', 'vendor_id', 'passenger_count', 'pickup_longitude', 'dropoff_longitude', 'trip_duration']

In [ ]:
MLlib classifiers and regressors require data sets in a format of rows of type LabeledPoint, which separates row labels and feature lists, and names them accordingly. 

In [5]:
from pyspark.mllib.linalg import Vectors
from pyspark.mllib.regression import LabeledPoint
from pyspark.mllib.tree import DecisionTree, DecisionTreeModel
from pyspark.mllib.util import MLUtils

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [6]:
#rdd = df.rdd.map(list)
from pyspark.sql.types import StringType
from pyspark.sql.types import DoubleType
from pyspark.sql.functions import udf
from pyspark.sql.functions import lit,unix_timestamp
addString = udf(lambda x: x[2:] , StringType())

trainingData=trainingData.withColumn("id",addString(trainingData["id"]))
#test_df=test_df.withColumn("id",addString(test_df["id"]))
#df = df.withColumn('pickup_datetime',unix_timestamp(df["pickup_datetime"],'yyyy-MM-dd HH:mm:ss').cast("timestamp"))
#df = df.withColumn('dropoff_datetime',unix_timestamp(df["dropoff_datetime"],'yyyy-MM-dd HH:mm:ss').cast("timestamp"))
trainingData = trainingData.withColumn('pickup_longitude',trainingData['pickup_longitude'].cast(DoubleType()))
trainingData = trainingData.withColumn('dropoff_longitude',trainingData['dropoff_longitude'].cast(DoubleType()))
#testData = testData.withColumn('pickup_longitude',test_df['pickup_longitude'].cast(DoubleType()))
#testData = testData.withColumn('dropoff_longitude',test_df['dropoff_longitude'].cast(DoubleType()))
t_df = trainingData.rdd.map(lambda row: LabeledPoint(row[-1], Vectors.dense(row[:-1])))
#testData2 = test_df.rdd.map(lambda row: LabeledPoint(row[-1], Vectors.dense(row[0:-1])))
trainingData1, testData1  =t_df.randomSplit([0.8,0.2])
t_df.take(4)


VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

[LabeledPoint(455.0, [2875421.0,2.0,1.0,-73.9821548461914,-73.96463012695312]), LabeledPoint(663.0, [2377394.0,1.0,1.0,-73.98041534423828,-73.99948120117188]), LabeledPoint(2124.0, [3858529.0,2.0,1.0,-73.9790267944336,-74.00533294677734]), LabeledPoint(429.0, [3504673.0,2.0,1.0,-74.01004028320312,-74.01226806640625])]

In [ ]:
I trained the model below with maxDepth = 10 and MaxBins = 6

In [8]:
model = DecisionTree.trainRegressor(trainingData1,categoricalFeaturesInfo={},
                                    impurity='variance', maxDepth=10, maxBins=6)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
model.toDebugString()

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

'DecisionTreeModel regressor of depth 10 with 1245 nodes\n  If (feature 3 <= -73.95919799804688)\n   If (feature 4 <= -73.95524215698242)\n    If (feature 1 <= 1.5)\n     If (feature 3 <= -73.99696731567383)\n      If (feature 4 <= -73.98733901977539)\n       If (feature 4 <= -73.99589920043945)\n        If (feature 2 <= 1.5)\n         If (feature 0 <= 2668437.0)\n          If (feature 2 <= 0.5)\n           If (feature 0 <= 1322089.0)\n            Predict: 13.0\n           Else (feature 0 > 1322089.0)\n            Predict: 2.0\n          Else (feature 2 > 0.5)\n           If (feature 0 <= 1322089.0)\n            Predict: 539.615838150289\n           Else (feature 0 > 1322089.0)\n            Predict: 529.4491086831513\n         Else (feature 0 > 2668437.0)\n          If (feature 2 <= 0.5)\n           If (feature 0 <= 3303608.0)\n            Predict: 68.0\n           Else (feature 0 > 3303608.0)\n            Predict: 1121.0\n          Else (feature 2 > 0.5)\n           If (feature 0 <= 3

In [10]:
from pyspark.mllib.evaluation import RegressionMetrics
#predictions = model.predict(testData.foreach(lambda x: print(x)))
predictions = model.predict(testData1.map(lambda x: x.features))

labels = testData1.map(lambda lp: lp.label)
print(len(predictions.collect()))
print(len(labels.collect()))
#labels = [ s.encode('ascii', 'ignore') for s in labels.collect()]
result  = [((lp[0] - lp[1]) * (lp[0] - lp[1])) for lp in zip(labels.collect(),predictions.collect())]
#labelsandPredictions = predictions.zip(labels)   291816
#print(len(labelsandPredictions.collect()))
#labelsAndPredictions = testData1.map(lada lp: lp.label).zip(predictions)
#metrics = RegressionMetrics(labelsAndPredictions)
#print(metrics.meanSquaredError)
#printMetrics(predictions_and_labels)
#labelsAndPredictions = testData.map(lambda lp: lp.label).zip(predictions)
#print(labelsAndPredictions)
#labelsAndPredictions.foreach(lambda x: print(x))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

292331
292331

In [ ]:
I got the following result as the Mean square error(MSE) :

In [16]:
print(sum(result)/len(testData1.collect()))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

10712028.350182772

In [17]:
print(testData1.count())

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

292331